# Minimal Cut Sets


## Example Network

In [14]:
import straindesign as sd
import cobra
model = cobra.io.read_sbml_model('../../../tests/model_small_example.xml')

modules =   [sd.SDModule(model,sd.names.PROTECT, constraints='R3 >= 1')]
modules +=  [sd.SDModule(model,sd.names.SUPPRESS,constraints='R4 <= 1')]

sols = sd.compute_strain_designs(model,sd_modules = modules)
for s in sols:
    print(s.sd_solutions)

TypeError: 'SDSolution' object is not iterable

## Computing synthetic lethals

## Computing production strain designs